## Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.utils import resample
import xgboost as xgb
import itertools
from matplotlib import pyplot as plt
import seaborn as sns

def cb(series):
    return series.apply(lambda x: 1 if x > 0 else 0)

def bootstrap_ci(y_true, y_pred, n_bootstraps=2000, alpha=0.05):
    bootstrapped_scores = {'auroc': [], 'auprc': []}
    indices = np.arange(len(y_true))
    for _ in range(n_bootstraps):
        sampled_indices = resample(indices, replace=True)
        bootstrapped_scores['auroc'].append(roc_auc_score(y_true[sampled_indices], y_pred[sampled_indices]))
        bootstrapped_scores['auprc'].append(average_precision_score(y_true[sampled_indices], y_pred[sampled_indices]))

    ci = {}
    for score_type in bootstrapped_scores:
        sorted_scores = np.sort(bootstrapped_scores[score_type])
        lower = np.percentile(sorted_scores, 100 * alpha / 2)
        upper = np.percentile(sorted_scores, 100 * (1 - alpha / 2))
        mean = np.mean(sorted_scores)
        ci[score_type] = (lower, mean, upper)
    return ci

cov = ['loeuf'] 
ot = ['clin_ot','hgmd','ot_genetics_portal','expression_atlas','impc','europepmc']
mantis = ['mantis']
cc = ['cc_common_max_p','cc_rare_max_p','cc_rare_burden_max_p','cc_ultrarare_max_p']

## Disclaimer

The files referenced (including "ml_input.pkl") are not currently available but will be released upon publication. This file will contain cleaned data for all phecodes in phecodeX.

## Preparing files

We first read the dataframe containing input data and select phecodes of interest.

In [ ]:
alldata = pd.read_pickle('./Examples/ml_input.pkl')

if False:
    phecode_list = [] # Selected phecodes
    alldata = alldata.loc[alldata['phecode'].isin(phecode_list)]

We train RareGPS using a logistic regression ("reg:logistic") XGBoost model where we assign true values probabilities from 0 to 1 based on the maximum clinical trial phase for the G-P pair.

In [ ]:
alldata['xscore'] = 0.0
alldata.loc[alldata['phase'] == 0.5, 'xscore'] = 1*0.732*0.548*0.580*0.911
alldata.loc[alldata['phase'] == 1, 'xscore'] = 1*0.732*0.548*0.580
alldata.loc[alldata['phase'] == 2, 'xscore'] = 1*0.732*0.548
alldata.loc[alldata['phase'] == 3, 'xscore'] = 1*0.732
alldata.loc[alldata['phase'] == 4, 'xscore'] = 1

We recommend training RareGPS only on G-P pairs representing druggable genes ('drug_gene' == 1) and phecodes with at least one drug indication ('indication' == 1).

In [ ]:
indata = alldata.loc[alldata['drug_gene'] == 1]
ind = indata.groupby('phecode')['indication'].max().reset_index()
ind = ind.loc[ind['indication'] == 1]
indata = indata.loc[indata['phecode'].isin(ind['phecode'])]
print(indata['phecode'].nunique())

Here we generate separate training and external testing sets.

In [ ]:
train_set = indata.loc[indata['source'].astype(str) != "['Minikel']"]
test_set = indata.loc[(indata['source'].astype(str).str.contains('Minikel')) | (indata['indication'] == 0)]

ind = train_set.groupby('phecode')['indication'].max().reset_index()
ind = ind.loc[ind['indication'] == 1]
train_set = train_set.loc[train_set['phecode'].isin(ind['phecode'])]
print(train_set['phecode'].nunique())

ind = test_set.groupby('phecode')['indication'].max().reset_index()
ind = ind.loc[ind['indication'] == 1]
test_set = test_set.loc[test_set['phecode'].isin(ind['phecode'])]
print(test_set['phecode'].nunique())

We also wish to generate predictions for all protein coding genes where at least one of the features is nonzero.

In [ ]:
apc = alldata.loc[alldata[ot+mantis+cc].max(axis=1) > 0][['id','gene','phecode','indication','phase','xscore']+cov+ot+mantis+cc]

## Hyperparameter tuning

Here we perform hyperparameter tuning and examine AUROC and AUPRC for holdout set predictions.

In [ ]:
params_test = {
    'objective': ['reg:logistic'],
    'booster': ['gbtree'],
    'max_depth': [5,6,7],
    'alpha': [1,2,5,7,10,20],
    'lambda': [1,2,5,7,10],
    'min_child_weight': [1,2,5,7,10,20],
    'subsample': [1],
    'colsample_bytree': [1],
    'nthread': [16]
}
keys, values = zip(*params_test.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
df = pd.DataFrame(combinations)
param_list = df.sample(n=40).to_dict(orient='records')
pi = 0

mnum = 0 # Select model
model_names = [['RareGPS_No_GA','RareGPS'][mnum]]
feature_names = [[cov+ot+mantis,cov+ot+mantis+cc][mnum]]

for i,j in zip(model_names, feature_names):
    
    metrics = []
    
    for params in param_list:
        print(pi)
        pi+=1
        
        X = train_set[j]
        y = train_set['xscore']
        ids = train_set['id']
        
        outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
        
        holdout_predictions = []
        
        for fold, (train_val_idx, holdout_idx) in enumerate(outer_cv.split(X), 1):
            print(f"Fold {fold}")
            
            X_train_val, X_holdout = X.iloc[train_val_idx], X.iloc[holdout_idx]
            y_train_val, y_holdout = y.iloc[train_val_idx], y.iloc[holdout_idx]
            ids_train_val, ids_holdout = ids.iloc[train_val_idx], ids.iloc[holdout_idx]
            
            inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)
            
            for inner_fold, (train_idx, val_idx) in enumerate(inner_cv.split(X_train_val), 1):
                print(f"  Inner Fold {inner_fold}")
                
                X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
                y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]
                ids_train, ids_val = ids_train_val.iloc[train_idx], ids_train_val.iloc[val_idx]
                
                train_data = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
                val_data = xgb.DMatrix(X_val, label=y_val, enable_categorical=True)
                
                evals = [(train_data, 'train'), (val_data, 'eval')]
                model = xgb.train(params, train_data, num_boost_round=3000, early_stopping_rounds=10, evals=evals, verbose_eval=False)
             
                y_pred_hold = model.predict(xgb.DMatrix(X_holdout, enable_categorical=True))
                auroc_hold = roc_auc_score(cb(y_holdout), y_pred_hold)
                auprc_hold = average_precision_score(cb(y_holdout), y_pred_hold)
                print(auroc_hold, auprc_hold)
        
                fold_predictions = pd.DataFrame({
                    'id': ids_holdout,
                    'prediction': y_pred_hold,
                    'outer_fold': fold,
                    'inner_fold': inner_fold
                })
                holdout_predictions.append(fold_predictions)
    
                filtered_params = {k: params[k] for k in ['max_depth', 'alpha', 'lambda', 'min_child_weight', 'subsample','colsample_bytree']}
                metrics_dict = {'Outer fold': fold, 'Inner fold': inner_fold,
                                'AUROC_hold': auroc_hold, 'AUPRC_hold': auprc_hold}
                metrics_dict = {**filtered_params, **metrics_dict}
                metrics.append(metrics_dict)
                print(metrics_dict)
    
    metrics_df = pd.DataFrame(metrics)
    metrics_df.to_pickle(f'./Outputs/Tuning/holdout_tuning_{i}.pkl')

## Train RareGPS

Using optimal hyperparameters we can then train RareGPS.

In [ ]:
mnum = 0 # Select model
model_names = [['RareGPS_No_GA','RareGPS'][mnum]]
feature_names = [[cov+ot+mantis,cov+ot+mantis+cc][mnum]]

for i,j in zip(model_names, feature_names):
    print(i)
    
    X = train_set[j]
    y = train_set['xscore']
    ids = train_set['id']

    holdout_predictions = []    
    ext_predictions = []
    apc_predictions = pd.DataFrame()
    
    outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
    
    for fold, (train_val_idx, holdout_idx) in enumerate(outer_cv.split(X), 1):
        print(f"Fold {fold}")
        
        X_train_val, X_holdout = X.iloc[train_val_idx], X.iloc[holdout_idx]
        y_train_val, y_holdout = y.iloc[train_val_idx], y.iloc[holdout_idx]
        ids_train_val, ids_holdout = ids.iloc[train_val_idx], ids.iloc[holdout_idx]

        apc_predictions_inner = []
        
        inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)
        
        for inner_fold, (train_idx, val_idx) in enumerate(inner_cv.split(X_train_val), 1):
            print(f"  Inner Fold {inner_fold}")
            
            X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
            y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]
            ids_train, ids_val = ids_train_val.iloc[train_idx], ids_train_val.iloc[val_idx]
            
            ext = test_set.loc[~((test_set['id'].isin(ids_train)) | (test_set['id'].isin(ids_val)))]
            X_ext = ext[j]
            y_ext = ext['xscore']
            ids_ext = ext['id']
            
            train_data = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
            val_data = xgb.DMatrix(X_val, label=y_val, enable_categorical=True)

            # Adjust based on hyperparameter tuning
            params = {'objective': 'reg:logistic','booster': 'gbtree','max_depth': 7,'alpha': 5,'lambda': 7,'min_child_weight': 1,'nthread': 16}

            evals = [(train_data, 'train'), (val_data, 'eval')]
            model = xgb.train(params, train_data, num_boost_round=3000, early_stopping_rounds=15, evals=evals, verbose_eval=False)
            model.save_model(f"./Outputs/New models/{i}_{fold}_{inner_fold}.json")
            
            # Generate predictions for holdout set
            y_pred_hold = model.predict(xgb.DMatrix(X_holdout, enable_categorical=True))
            auroc_hold = roc_auc_score(cb(y_holdout), y_pred_hold)
            auprc_hold = average_precision_score(cb(y_holdout), y_pred_hold)
            print('Holdout', auroc_hold, auprc_hold)

            fold_predictions = pd.DataFrame({
                'id': ids_holdout,
                'prediction': y_pred_hold
            })
            holdout_predictions.append(fold_predictions)

            # Generate predictions for external test set
            y_pred_ext = model.predict(xgb.DMatrix(X_ext, enable_categorical=True))
            auroc_ext = roc_auc_score(cb(y_ext), y_pred_ext)
            auprc_ext = average_precision_score(cb(y_ext), y_pred_ext)
            print('External', auroc_ext, auprc_ext)

            fold_predictions = pd.DataFrame({
                'id': ids_ext,
                'prediction': y_pred_ext
            })
            ext_predictions.append(fold_predictions)

            # Generate predictions for all protein coding genes
            apc_subset = apc.loc[apc['id'].isin(ids_train_val)]
            y_pred_apc = model.predict(xgb.DMatrix(apc_subset[j], enable_categorical=True))
            
            fold_predictions = pd.DataFrame({
                'id': apc_subset['id'],
                'prediction': y_pred_apc
            })
            apc_predictions_inner.append(fold_predictions)            

        apc_predictions_inner = pd.concat(apc_predictions_inner, ignore_index=True)
        apc_predictions_inner = apc_predictions_inner.groupby('id')['prediction'].mean().reset_index()
        apc_predictions = pd.concat([apc_predictions,apc_predictions_inner])

    holdout_predictions = pd.concat(holdout_predictions, ignore_index=True)
    holdout_predictions = holdout_predictions.groupby('id')['prediction'].mean().reset_index()
    all_predictions = holdout_predictions.merge(indata[['id','gene','phecode','indication','phase','xscore']])
    all_predictions.to_pickle(f'./Outputs/Predictions/holdout_predictions_{i}.pkl')

    ext_predictions = pd.concat(ext_predictions, ignore_index=True)
    ext_predictions = ext_predictions.groupby('id')['prediction'].mean().reset_index()
    all_predictions = ext_predictions.merge(indata[['id','gene','phecode','indication','phase','xscore']])
    all_predictions.to_pickle(f'./Outputs/Predictions/ext_predictions_{i}.pkl')

    all_predictions = pd.concat([holdout_predictions,ext_predictions]).drop_duplicates(['id']).reset_index(drop=True)
    all_predictions = all_predictions.merge(indata[['id','gene','phecode','indication','phase','xscore']])
    all_predictions.to_pickle(f'./Outputs/Predictions/all_predictions_{i}.pkl')

    apc_predictions =  apc_predictions.merge(apc[['id','gene','phecode','indication','phase','xscore']])
    apc_predictions.to_pickle(f'./Outputs/Predictions/apc_predictions_{i}.pkl')


## Calculate and plot metrics

In [ ]:
conds = ['RareGPS_No_GA','RareGPS']

results = []
for cond in conds:
    print('Holdout',cond)
    pdf = pd.read_pickle(f'./Outputs/Predictions/holdout_predictions_{cond}.pkl').reset_index(drop=True)
    auroc = roc_auc_score(pdf['indication'], pdf['prediction'])
    auprc = average_precision_score(pdf['indication'], pdf['prediction'])
    ci = bootstrap_ci(pdf['indication'], pdf['prediction'])
    results.append({'Dataset':'Holdout','Features':cond,
                    'AUROC':auroc, 'AUROC_CI': ci['auroc'],
                    'AUPRC':auprc, 'AUPRC_CI': ci['auprc']})
res_df = pd.DataFrame(results)
res_df.to_excel('./Results/bootstrap_metrics_holdout.xlsx', index=False)

results = []
for cond in conds:
    print('Ext',cond)
    pdf = pd.read_pickle(f'./Outputs/Predictions/ext_predictions_{cond}.pkl').reset_index(drop=True)
    auroc = roc_auc_score(pdf['indication'], pdf['prediction'])
    auprc = average_precision_score(pdf['indication'], pdf['prediction'])
    ci = bootstrap_ci(pdf['indication'], pdf['prediction'])
    results.append({'Dataset':'External','Features':cond,
                    'AUROC':auroc, 'AUROC_CI': ci['auroc'],
                    'AUPRC':auprc, 'AUPRC_CI': ci['auprc']})
res_df = pd.DataFrame(results)
res_df.to_excel('./Results/bootstrap_metrics_ext.xlsx', index=False)

results = []
for cond in conds:
    print('All',cond)
    pdf = pd.read_pickle(f'./Outputs/Predictions/all_predictions_{cond}.pkl').reset_index(drop=True)
    auroc = roc_auc_score(pdf['indication'], pdf['prediction'])
    auprc = average_precision_score(pdf['indication'], pdf['prediction'])
    ci = bootstrap_ci(pdf['indication'], pdf['prediction'])
    results.append({'Dataset':'Combined','Features':cond,
                    'AUROC':auroc, 'AUROC_CI': ci['auroc'],
                    'AUPRC':auprc, 'AUPRC_CI': ci['auprc']})
res_df = pd.DataFrame(results)
res_df.to_excel('./Results/bootstrap_metrics_all.xlsx', index=False)

In [ ]:
a = pd.read_excel('./Results/bootstrap_metrics_holdout.xlsx')
b = pd.read_excel('./Results/bootstrap_metrics_ext.xlsx')
c = pd.read_excel('./Results/bootstrap_metrics_all.xlsx')
metrics = pd.concat([a,b,c])

In [ ]:
df = metrics.copy()
df['Dataset'] = pd.Categorical(df['Dataset'], ['Holdout','External','Combined'])
df = df.sort_values(['Dataset','Model'])

df['AUROC_mean'] = df['AUROC_CI'].apply(lambda x: ast.literal_eval(x)).apply(lambda x: x[1]).astype(float)
df['AUROC_CI_lower'] = df['AUROC_CI'].apply(lambda x: ast.literal_eval(x)).apply(lambda x: x[0]).astype(float)
df['AUROC_CI_upper'] = df['AUROC_CI'].apply(lambda x: ast.literal_eval(x)).apply(lambda x: x[2]).astype(float)
df['error_lower'] = df['AUROC_mean'] - df['AUROC_CI_lower']
df['error_upper'] = df['AUROC_CI_upper'] - df['AUROC_mean']

plt.figure(figsize=(8, 4), dpi=300)
sns.set_style('white')

datasets = df['Dataset'].unique()
models = df['Model'].unique()

x = np.arange(len(datasets))
width = 0.175 

for i, model in enumerate(models):
    model_data = df[df['Model'] == model]
    plt.errorbar(x=x + (i - (len(models) - 1) / 2) * width, 
                 y=model_data['AUROC_mean'], 
                 yerr=[model_data['error_lower'], model_data['error_upper']], 
                 fmt='o', capsize=0, label=model, color=CB_color_cycle[i])

plt.xticks(ticks=x, labels=datasets)
plt.ylabel('AUROC')

In [ ]:
hold_prop = pd.read_pickle(f'./ML/Predictions/holdout_predictions_cov.pkl')['indication'].mean()
ext_prop = pd.read_pickle(f'./ML/Predictions/ext_predictions_cov.pkl')['indication'].mean()
all_prop = pd.read_pickle(f'./ML/Predictions/all_predictions_cov.pkl')['indication'].mean()

df = metrics.loc[~metrics['Features'].isin(['cov_otnopmc','cov_otnopmc_cc'])]
df['Proportion'] = df['Dataset'].map({'Holdout':hold_prop,'External':ext_prop,'Combined':all_prop})
df['Dataset'] = pd.Categorical(df['Dataset'], ['Holdout','External','Combined'])
df = df.sort_values(['Dataset','Model'])

df['AUPRC_mean'] = df['AUPRC_CI'].apply(lambda x: ast.literal_eval(x)).apply(lambda x: x[1]).astype(float)
df['AUPRC_CI_lower'] = df['AUPRC_CI'].apply(lambda x: ast.literal_eval(x)).apply(lambda x: x[0]).astype(float)
df['AUPRC_CI_upper'] = df['AUPRC_CI'].apply(lambda x: ast.literal_eval(x)).apply(lambda x: x[2]).astype(float)
df['error_lower'] = df['AUPRC_mean'] - df['AUPRC_CI_lower']
df['error_upper'] = df['AUPRC_CI_upper'] - df['AUPRC_mean']

plt.figure(figsize=(8, 4), dpi=300)
sns.set_style('white')

datasets = df['Dataset'].unique()
models = df['Model'].unique()

x = np.arange(len(datasets))
width = 0.175 

for i, model in enumerate(models):
    model_data = df[df['Model'] == model]
    plt.errorbar(x=x + (i - (len(models) - 1) / 2) * width, 
                 y=model_data['AUPRC_mean'], 
                 yerr=[model_data['error_lower'], model_data['error_upper']], 
                 fmt='o', capsize=0, label=model, color=CB_color_cycle[i])

for i, dataset in enumerate(datasets):
    proportion = df[df['Dataset'] == dataset]['Proportion'].iloc[0]
    left = (x[i] - (len(models) - 1) / 2 * width)
    right = (x[i] + (len(models) - 1) / 2 * width)
    plt.hlines(y=proportion, xmin=left, xmax=right, color='gray', linestyle='-', linewidth=1)


plt.xticks(ticks=x, labels=datasets)
plt.ylabel('AUPRC')